In [14]:
# Implementing Decision Tree Algorithm from scratch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [72]:
# Loading the iris dataset
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
iris = pd.read_csv(url)

In [16]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [17]:
iris.tail()

,sepal_length,sepal_width,petal_length,petal_width,species
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica


In [18]:
iris.shape

(150, 5)

In [19]:
# Counts the number of null values, This is used to check if cleaning is required
iris.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [20]:
# Gives details about distribution of data
iris.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [21]:
# This is similar to describe, but here we group by species
metrics = ['count', 'min', 'max', 'mean','std','skew']
iris.groupby(by='species').agg(metrics)

sepal_length                                      sepal_width       \
                  count  min  max   mean       std      skew       count  min   
species                                                                         
setosa               50  4.3  5.8  5.006  0.352490  0.120087          50  2.3   
versicolor           50  4.9  7.0  5.936  0.516171  0.105378          50  2.0   
virginica            50  4.9  7.9  6.588  0.635880  0.118015          50  2.2   

                        ... petal_length                             \
            max   mean  ...          max   mean       std      skew   
species                 ...                                           
setosa      4.4  3.428  ...          1.9  1.462  0.173664  0.106394   
versicolor  3.4  2.770  ...          5.1  4.260  0.469911 -0.606508   
virginica   3.8  2.974  ...          6.9  5.552  0.551895  0.549445   

           petal_width                                       
                 count  min  max   mean       std      skew  
species                                                      
setosa              50  0.1  0.6  0.246  0.105386  1.253861  
versicolor          50  1.0  1.8  1.326  0.197753 -0.031180  
virginica           50  1.4  2.5  2.026  0.274650 -0.129477  

[3 rows x 24 columns]

In [73]:
# Assigning class labels to the data
iris["species"] = iris["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2})
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [142]:
# Defining a train and test split
test_size = int(0.0 * len(iris))
test_indices = iris.sample(test_size, random_state=42).index
train = iris.drop(test_indices)
test = iris.loc[test_indices]
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
print(train.shape, test.shape)

(150, 5) (0, 5)


In [143]:
labels = train.pop("species").tolist()

In [144]:
print(len(labels))

150


In [145]:
# Node class represents a node in the decision tree
class Node:
    def __init__(self, label=-1, purity=-1, split_idx=-1, split_val=-1, left=None, right=None):
        self.label = label
        self.purity = purity
        self.split_idx = split_idx
        self.split_val = split_val
        self.left = left
        self.right = right

In [146]:
def check_purity(sample):
    count = [0, 0, 0]
    for i in range(len(sample)):
        count[labels[sample[i]]] += 1
    
    return (count.index(max(count)), max(count) / len(sample))

In [147]:
# BuildTree is used to build the decision tree in a recursive fashion. We use two parameters, sample and threshold. sample is the list of all data instances in the current node. threshold is the purity threshold. If the purity of the current node is greater than threshold, we create a leaf node.

def BuildTree(sample, threshold = 0.9): 
    purity = check_purity(sample)

    # If the purity of the current node is greater than threshold, we create a leaf node.
    if (purity[1] >= threshold):
        print("Leaf node: ", purity[0], purity[1])
        return Node(label=purity[0], purity=purity[1])
    
    # If the purity of the current node is less than threshold, we split the node into two child nodes. For this we require split_idx and split_val.
    else:
        split_idx = -1
        split_val = -1
        node_purity = -1
        left_sample = []
        right_sample = []
        new_iris = train[train.index.isin(sample)].copy()

        # We traverse through each feature to find the best split index
        for i in range(new_iris.shape[1]):
            col = new_iris.columns[i]
            new_iris.sort_values(by=[col], inplace=True)
            unique = new_iris[col].unique()

            # For each data instance, we try to find the best split value
            for j in range(1,len(unique)):
                temp_left_sample = []
                temp_right_sample = []
                temp_split_val = (unique[j-1] + unique[j]) / 2

                for k in range(len(sample)):
                    if (train.iloc[sample[k], i] <= temp_split_val):
                        temp_left_sample.append(sample[k])
                    else:
                        temp_right_sample.append(sample[k])

                purity_left = check_purity(temp_left_sample)
                purity_right = check_purity(temp_right_sample)
                net = (purity_left[0] + purity_right[0]) / 2
                
                if (net > node_purity):
                    node_purity = net
                    split_idx = i
                    split_val = temp_split_val
                    left_sample = temp_left_sample
                    right_sample = temp_right_sample

        print("Split index: ", split_idx)
        print("Split value: ", split_val)
        print(len(left_sample), len(right_sample))
        left_node = BuildTree(left_sample)
        right_node = BuildTree(right_sample)
        return Node(split_idx=split_idx, split_val=split_val, left=left_node, right=right_node)

In [148]:
sample = [i for i in range(train.shape[0])]
model = BuildTree(sample)

Split index:  0
Split value:  5.05
32 118
Split index:  2
Split value:  4.0
31 1
Leaf node:  0 0.9032258064516129
Leaf node:  2 1.0
Split index:  0
Split value:  7.800000000000001
117 1
Split index:  0
Split value:  5.95
51 66
Split index:  2
Split value:  4.85
45 6
Split index:  0
Split value:  5.15
9 36
Split index:  1
Split value:  2.9
1 8
Leaf node:  1 1.0
Leaf node:  0 1.0
Split index:  0
Split value:  5.65
23 13
Split index:  0
Split value:  5.25
4 19
Split index:  1
Split value:  3.05
1 3
Leaf node:  1 1.0
Leaf node:  0 1.0
Split index:  1
Split value:  2.3499999999999996
1 18
Leaf node:  1 1.0
Split index:  0
Split value:  5.35
1 17
Leaf node:  0 1.0
Split index:  1
Split value:  2.45
2 15
Leaf node:  1 1.0
Split index:  0
Split value:  5.45
6 9
Split index:  1
Split value:  3.2
1 5
Leaf node:  1 1.0
Leaf node:  0 1.0
Split index:  1
Split value:  2.55
2 7
Leaf node:  1 1.0
Split index:  1
Split value:  2.6500000000000004
1 6
Leaf node:  1 1.0
Split index:  1
Split value:  2.8
